In [1]:
!pip install mtranslate==1.8

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mtranslate as mt

# Employment

In [4]:
def trimester_to_date(trimester):
    year, quarter = trimester.split('-')
    quarter=quarter[1]
    month = int(quarter) * 3
    return pd.Timestamp(year=int(year), month=month, day=1)

In [14]:
def translate_country_name(text):
    return mt.translate(text, 'en', 'fr')

In [6]:
employment=pd.read_csv('Global_original/employment.csv')

In [7]:
employment.columns

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'FREQ',
       'Fréquence d'observation', 'ADJUSTMENT', 'Ajustement', 'REF_AREA',
       'Zone de référence', 'SECTOR', 'Secteur institutionnel',
       'COUNTERPART_SECTOR', 'Secteur institutionnel de contrepartie',
       'TRANSACTION', 'Transaction', 'INSTR_ASSET',
       'Instruments financiers et actifs non financiers', 'ACTIVITY',
       'Activité économique', 'EXPENDITURE', 'Dépense', 'UNIT_MEASURE',
       'Unité de mesure', 'PRICE_BASE', 'Type de prix', 'TRANSFORMATION',
       'Transformation', 'TABLE_IDENTIFIER', 'Identifiant de tableau',
       'TIME_PERIOD', 'Période temporelle', 'OBS_VALUE',
       'Valeur d'observation', 'REF_YEAR_PRICE', 'Prix ​​année de référence',
       'BASE_PER', 'Période de base', 'CONF_STATUS',
       'Statut de confidentialité', 'DECIMALS', 'Décimales', 'OBS_STATUS',
       'Statut d'observation', 'UNIT_MULT', 'Multiplicateur d'unité',
       'CURRENCY', 'Monnaie'],
      dtype='obj

In [8]:
employment=employment[employment['UNIT_MEASURE']=='H']
employment=employment[employment['Transaction']=='Emploi total']
employment=employment[employment['ADJUSTMENT']=='Y']


In [9]:
employment2=employment.drop(['STRUCTURE','STRUCTURE_ID','STRUCTURE_NAME','ACTION','FREQ',
       'ADJUSTMENT', 'Ajustement', 'REF_AREA','SECTOR','Secteur institutionnel','COUNTERPART_SECTOR','Secteur institutionnel de contrepartie',
       'TRANSACTION','INSTR_ASSET','Instruments financiers et actifs non financiers','ACTIVITY','EXPENDITURE','Dépense','UNIT_MEASURE','PRICE_BASE',
       'Type de prix','TRANSFORMATION','Transformation','TABLE_IDENTIFIER','Identifiant de tableau','Période temporelle','Valeur d\'observation',
       'REF_YEAR_PRICE','Prix ​​année de référence','BASE_PER','Période de base','CONF_STATUS','Statut de confidentialité','DECIMALS','Décimales',
       'OBS_STATUS','Statut d\'observation','UNIT_MULT','CURRENCY','Monnaie'],axis=1)

In [10]:
employment2.columns

Index(['Fréquence d'observation', 'Zone de référence', 'Transaction',
       'Activité économique', 'Unité de mesure', 'TIME_PERIOD', 'OBS_VALUE',
       'Multiplicateur d'unité'],
      dtype='object')

In [11]:
employment2.TIME_PERIOD=employment2.TIME_PERIOD.astype(str)
employment2=employment2[employment2['Fréquence d\'observation']=='Trimestrielle']
employment2.OBS_VALUE=employment2.OBS_VALUE.astype(float)


In [12]:
for country in employment2['Zone de référence'].unique():
    for activite in employment2['Activité économique'].unique():
        a=employment2[(employment2['Zone de référence']==country) & (employment2['Activité économique']==activite)]['OBS_VALUE'].pct_change()
        employment2.loc[(employment2['Zone de référence']==country) & (employment2['Activité économique']==activite),'OBS_VALUE']=a

employment2=employment2.dropna(subset='OBS_VALUE')
employment2.loc[:,'TIME_PERIOD']=employment2.TIME_PERIOD.apply(lambda x: trimester_to_date(x))
employment2['TIME_PERIOD']=pd.to_datetime(employment2['TIME_PERIOD'])

In [15]:
translate_dic = {}
for country in list(employment2['Zone de référence'].unique()):
    translate_dic[country] = translate_country_name(country)
    
    
employment2['Zone de référence'] = employment2['Zone de référence'].replace(translate_dic)

In [17]:
employment2.to_csv('global/dataglobal/clean_employment.csv')

# Employee Compensation

In [18]:
compensation=pd.read_csv('Global_original/employeecompensation.csv')

In [19]:
compensation=compensation[compensation['FREQ']=='Q']
compensation=compensation[compensation['ADJUSTMENT']=='Y']


In [20]:
compensation2=compensation.drop(['STRUCTURE','STRUCTURE_ID','ACTION','FREQ','Fréquence d\'observation','ADJUSTMENT','Ajustement',
                                 'REF_AREA','SECTOR','COUNTERPART_SECTOR','Secteur institutionnel de contrepartie','TRANSACTION','INSTR_ASSET','Instruments financiers et actifs non financiers',
                                 'ACTIVITY','EXPENDITURE','Dépense','UNIT_MEASURE','Unité de mesure','PRICE_BASE','Type de prix','TRANSFORMATION','Transformation',
                                 'TABLE_IDENTIFIER','Identifiant de tableau','Période temporelle','Valeur d\'observation','REF_YEAR_PRICE','Prix ​​année de référence',
                                 'BASE_PER','Période de base','CONF_STATUS','Statut de confidentialité','DECIMALS','Décimales','OBS_STATUS','Statut d\'observation',
                                 'UNIT_MULT','CURRENCY','Monnaie'],axis=1)

In [21]:
compensation2.TIME_PERIOD=compensation2.TIME_PERIOD.astype(str)
compensation2.OBS_VALUE=compensation2.OBS_VALUE.astype(float)

In [22]:
for country in compensation2['Zone de référence'].unique():
    for activite in compensation2[compensation2['Zone de référence']==country]['Activité économique'].unique():
        a=compensation2[(compensation2['Zone de référence']==country) & (compensation2['Activité économique']==activite)]['OBS_VALUE'].pct_change()
        compensation2.loc[(compensation2['Zone de référence']==country) & (compensation2['Activité économique']==activite),'OBS_VALUE']=a

compensation2=compensation2.dropna(subset='OBS_VALUE')
compensation2.loc[:,'TIME_PERIOD']=compensation2.TIME_PERIOD.apply(lambda x: trimester_to_date(x))
compensation2['TIME_PERIOD']=pd.to_datetime(compensation2['TIME_PERIOD'])

In [ ]:
translate_dic = {}
for country in list(compensation2['Zone de référence'].unique()):
    translate_dic[country] = translate_country_name(country)
    
    
compensation2['Zone de référence'] = compensation2['Zone de référence'].replace(translate_dic)

In [24]:
compensation2.to_csv('global/dataglobal/clean_employee_compensation.csv')